In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [61]:
df = pd.read_csv('simResult.csv')
df

,kernel,parameter,pipelineWidth,robSize,branchPredictor,l1Size,l1Associativity,l1ReplacementPolicy,l1Prefetcher,l2Size,...,dramReadAccesses,dramWriteAccesses,memAvgLat,commitSquashedInsts,memOrderViolations,dispatchRenameStalls,dispatchIssueStalls,iqFullEvents,dramTotalEnergy,dramAveragePower
0,compute,baseline,4,192,tournament,32KiB,4,NaN,NaN,256KiB,...,3.0,0.0,20208.00,3685.0,191.0,340.0,340.0,0.0,2.140462e+09,465.753000
1,compute,l1a_2,4,192,tournament,32KiB,2,NaN,NaN,256KiB,...,3.0,0.0,20208.00,3678.0,191.0,340.0,340.0,0.0,2.140466e+09,465.753971
2,compute,l1a_8,4,192,tournament,32KiB,8,NaN,NaN,256KiB,...,3.0,0.0,20208.00,3678.0,191.0,340.0,340.0,0.0,2.140463e+09,465.753345
3,compute,l1p_stride,4,192,tournament,32KiB,4,NaN,stride,256KiB,...,3.0,0.0,20208.00,3685.0,191.0,340.0,340.0,0.0,2.140462e+09,465.753000
4,compute,l1p_tagged,4,192,tournament,32KiB,4,NaN,tagged,256KiB,...,3.0,0.0,18750.00,3685.0,191.0,340.0,340.0,0.0,2.138983e+09,465.435439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,stream,pw_12,12,192,tournament,32KiB,4,NaN,NaN,256KiB,...,393221.0,133667.0,64992.00,46425.0,1386.0,13586327.0,5727.0,0.0,1.291047e+10,625.052254
80,stream,pw_8,8,192,tournament,32KiB,4,NaN,NaN,256KiB,...,393221.0,133675.0,47708.00,52684.0,1353.0,446339.0,6355.0,0.0,1.287461e+10,625.655272
81,stream,rob_128,4,128,tournament,32KiB,4,NaN,NaN,256KiB,...,393225.0,133671.0,62278.00,32973.0,1284.0,480308.0,3891.0,0.0,1.321277e+10,620.347678
82,stream,rob_256,4,256,tournament,32KiB,4,NaN,NaN,256KiB,...,393229.0,133671.0,68944.67,32708.0,1276.0,242442.0,3818.0,0.0,1.314141e+10,621.555681


In [62]:
df['l1Size'] = (
    df['l1Size']
    .replace({'32KiB': 32, '16KiB': 16, '64KiB': 64})
    .astype(int)
)

df['l2Size'] = (
    df['l2Size']
    .replace({'1MB': 1024, '4MB': 4096, '256KiB': 256})
    .astype(int)
)

C:\Users\kihsuakias\AppData\Local\Temp\ipykernel_24152\4265436806.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({'32KiB': 32, '16KiB': 16, '64KiB': 64})
C:\Users\kihsuakias\AppData\Local\Temp\ipykernel_24152\4265436806.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({'1MB': 1024, '4MB': 4096, '256KiB': 256})


In [63]:
df['l1Size'].unique(), df['l2Size'].unique()

(array([32, 16, 64]), array([ 256, 1024, 4096]))

In [64]:
plotDir = "figs"
os.makedirs(plotDir, exist_ok=True)

sns.set_theme(style="whitegrid", context="talk")

In [65]:
correlation_columns = [
    'ipc',
    'l1Size',
    'l2Size',
    'l1Associativity',
    'l2Associativity',
    'l1dAMATCycles',
    'l2AMATCycles',
    'l1dMisses',
    'l2Misses',
    'icacheStallCycles',
    'dramTotalEnergy',
    'branchMispredicted',
]

corr_df = df[correlation_columns]
correlation_matrix = corr_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(
    correlation_matrix,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    linewidths=0.5
)
plt.title('Correlation Matrix of Key Performance and Energy Metrics', fontsize=16, weight="bold")

filename = 'correlation_heatmap.png'
plt.savefig(os.path.join(plotDir, filename), dpi=300, bbox_inches='tight')
plt.close()


In [70]:
for kernel in df["kernel"].unique():
    kernel_df = df[
        (df['kernel'] == kernel) &
        (df['parameter'].astype(str).str.startswith('l2p_') | (df['parameter'] == 'baseline'))
    ]
    
    if kernel_df.empty:
        continue

    df['l2Prefetcher'] = df['l2Prefetcher'].fillna('Off')
    kernel_df.loc[kernel_df['l2Prefetcher'] == 'Off', 'l2AMATCycles'] *= 1.01
    
    plt.figure(figsize=(12, 7))
    
    sns.scatterplot(
        data=kernel_df,
        x='l2AMATCycles',
        y='ipc',
        hue='l2Prefetcher',
        style='l2Prefetcher',
        s=250,
        palette="deep"
    )
    
    plt.xlabel("L2 AMAT", fontsize=12)
    plt.ylabel("IPC (Instructions Per Cycle)", fontsize=12)
    plt.title(f"IPC vs. L2 AMAT for Kernel '{kernel}' with L2 Prefetchers",
              fontsize=14, weight="bold")
    
    plt.grid(axis="y", linestyle="--", alpha=0.6)
    plt.tight_layout()
    
    filename = f'ipc_vs_AMAT_L2Prefetcher_{kernel}.png'
    plt.savefig(os.path.join(plotDir, filename), dpi=300, bbox_inches='tight')
    plt.close()

In [85]:
miss_cols = ['l1dMisses', 'l2Misses']

fig, axes = plt.subplots(1, 2, figsize=(16, 7), sharey=True)

for ax, miss_col in zip(axes, miss_cols):
    sns.scatterplot(
        data=df,
        x=miss_col,
        y='icacheStallCycles',
        hue='kernel',
        style='kernel',
        s=120,
        alpha=0.7,
        palette='tab10',
        ax=ax
    )
    
    for kernel in df['kernel'].unique():
        kernel_df = df[df['kernel'] == kernel]
        sns.regplot(
            data=kernel_df,
            x=miss_col,
            y='icacheStallCycles',
            scatter=False,
            line_kws={'lw':2},
            ax=ax
        )
    
    ax.set_xlabel(f"{miss_col}")
    ax.set_ylabel("Instruction Cache Stall Cycles")
    ax.set_title(f"{miss_col} vs Stalls Across All Kernels")
    ax.grid(True, linestyle='--', alpha=0.5)

plt.tight_layout()
filename = f'cache_misses_vs_stalls_all_kernels.png'
plt.savefig(os.path.join(plotDir, filename), dpi=300, bbox_inches='tight')
plt.close()